In [1]:
import gzip
import io
import networkx as nx
import numpy as np
import pandas as pd
import random
import requests
from sklearn import cluster, manifold, metrics
import time
import warnings
warnings.simplefilter("ignore")

import graph2vec
from graph2vec.graph import _sparse_normalize_rows

embed_size = 32

In [2]:
def evalClusteringOnLabels(clusters, groupLabels, verbose=True):
    results = []
    results.append(metrics.adjusted_mutual_info_score(clusters, groupLabels))
    results.append(metrics.adjusted_rand_score(clusters, groupLabels))
    results.append(metrics.fowlkes_mallows_score(clusters, groupLabels))
    if verbose:
        print("adj. MI score:   {0:.2f}".format(results[0]))
        print("adj. RAND score: {0:.2f}".format(results[1]))
        print("F-M score:       {0:.2f}".format(results[2]))
    return np.array(results)

In [3]:
def make_snap():
    """
    Graph from university emails, clustered by departments
    Data from http://snap.stanford.edu/data/email-Eu-core.html
    Edge list Format
    """
    res = requests.get('http://snap.stanford.edu/data/email-Eu-core.txt.gz', verify=False)
    edges = gzip.GzipFile(fileobj=io.BytesIO(res.content))
    edges = pd.read_csv(io.StringIO(edges.read().decode()), header=None, sep=' ')
    edges.columns = ['src', 'dest']
    # cluster labels per node
    res = requests.get('http://snap.stanford.edu/data/email-Eu-core-department-labels.txt.gz', verify=False)
    labels = gzip.GzipFile(fileobj=io.BytesIO(res.content))
    labels = pd.read_csv(io.StringIO(labels.read().decode()), header=None, sep=' ')
    labels.columns = ['node', 'cluster']
    G = nx.Graph()
    G.add_edges_from([(t.src, t.dest) for t in edges.itertuples()])
    return G, labels.cluster


def cluster_graph(n_nodes, n_clusters, 
                  connections=1,
                  drop_pct=0.1):
    """
    Makes distinct complete subgraphs
        connected by random paths
        
    n_nodes (int): number of nodes
    n_clusters (int): number of clusters
        This is also the number of disjoint subgraphs
    connections (int): number of random connections 
        These join the disjoint subgraphs
    """
    div = int(n_nodes / n_clusters)
    subgraph_sizes = [div] * n_clusters
    # last cluster has remainder nodes
    subgraph_sizes[-1] = subgraph_sizes[-1] + (n_nodes % n_clusters)

    # Make G from disjoint subgraphs
    G = nx.complete_graph(subgraph_sizes[0])
    for i in range(1, len(subgraph_sizes)):
        G = nx.disjoint_union(G, nx.complete_graph(subgraph_sizes[i]))

    # connecting paths
    for i in range(connections):
        while True:
            c1, c2 = np.random.randint(n_nodes, size=2)
            if G.has_edge(c1, c2):
                continue
            G.add_edge(c1, c2)
            break

    # Drop random edges
    n_edges = len(G.edges)
    to_remove=random.sample(G.edges(),
                            k=int(n_edges * drop_pct))
    G.remove_edges_from(to_remove)
    
    # Generate labels
    labels = []
    for i in range(len(subgraph_sizes)):
        labels.append([i] * subgraph_sizes[i])
    labels = sum(labels, [])
    assert len(labels) == n_nodes, f"{labels}"
    assert len(set(labels)) == n_clusters, f"{labels}"
    return G, pd.Series(labels)

In [4]:
G, labels = make_snap()
# G, labels = cluster_graph(1_000, 30, 50, 0.05)

# Graph nodes should be ordered ints
if not (np.array(list(G.nodes)) == np.arange(len(list(G.nodes)))).all():
    raise ValueError("graph nodes are not ordered")

# pick up n_clusters automatically
n_clusters = labels.nunique()

# Gridsearch result table
res = pd.DataFrame(columns=['method', 'params', 'traintime', 
                            'MI', 'RAND', 'F-M'])

In [5]:
import umap

for N_NEIGHBORS in [5, 15]:
 for MIN_DIST in [0.001, 0.01]:
  for EMBED_SIZE in [16, 64]:
    start_t = time.time()
    ump = umap.UMAP(
        n_neighbors=N_NEIGHBORS,
        min_dist=MIN_DIST,
        metric='euclidean',
        n_components=EMBED_SIZE,
    )
    umpembed = ump.fit_transform(
        _sparse_normalize_rows(nx.adjacency_matrix(G))
    )
    umpagglo = cluster.AgglomerativeClustering(
        n_clusters=n_clusters, 
        affinity='cosine', 
        linkage='average'
    ).fit(umpembed).labels_

    x = evalClusteringOnLabels(umpagglo, labels)
    print("-------------------\n\n")

    res = res.append({
        'method': 'UMAP', 
        'params':{
            'n_neigbors': N_NEIGHBORS,
            'min_dist': MIN_DIST,
            'embed_size': EMBED_SIZE,
        }, 
        'traintime': time.time() - start_t, 
        'MI':x[0], 
        'RAND':x[1], 
        'F-M':x[2]}, 
        ignore_index=True)

C:\anaconda3\lib\site-packages\numba\typed\typeddict.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)
C:\anaconda3\lib\importlib\_bootstrap.py

adj. MI score:   0.65
adj. RAND score: 0.56
F-M score:       0.58
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.64
adj. RAND score: 0.56
F-M score:       0.58
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


...

C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.66
adj. RAND score: 0.58
F-M score:       0.60
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.65
adj. RAND score: 0.57
F-M score:       0.59
-------------------


adj. MI score:   0.66
adj. RAND score: 0.58
F-M score:       0.60
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


In [6]:
for N_NEIGHBORS in [1, 2, 5, 10, 20]:
  for EMBED_SIZE in [16, 64]:
    start_t = time.time()
    pcae = manifold.Isomap(
        n_components=EMBED_SIZE,
        n_neighbors=N_NEIGHBORS,
    )
    pcae = pcae.fit_transform(
        _sparse_normalize_rows(nx.adjacency_matrix(G)).todense()
    )
    pcae = cluster.AgglomerativeClustering(
        n_clusters=n_clusters, 
        affinity='cosine', 
        linkage='average'
    ).fit(pcae).labels_

    x = evalClusteringOnLabels(pcae, labels)
    print("-------------------\n\n")

    res = res.append({
        'method': 'ISOMAP', 
        'params':{
            'n_neigbors': N_NEIGHBORS,
            'embed_size': EMBED_SIZE,
        }, 
        'traintime': time.time() - start_t, 
        'MI':x[0], 
        'RAND':x[1], 
        'F-M':x[2]}, 
        ignore_index=True)

C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.28
adj. RAND score: 0.11
F-M score:       0.21
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.23
adj. RAND score: 0.10
F-M score:       0.21
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


...

C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.61
adj. RAND score: 0.51
F-M score:       0.53
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.59
adj. RAND score: 0.50
F-M score:       0.52
-------------------


adj. MI score:   0.58
adj. RAND score: 0.48
F-M score:       0.51
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


In [7]:
for WALKLEN in [8, 20, 60]:
 for EPOCH in [7, 30]:
  for RETURN_WEIGHT in [0.3, 1., 2.]:
   for NEIGHBOR_WEIGHT in [0.3, 1., 2.]:
    for WINDOW in [3, 8]:
     for EMBED_SIZE in [16, 64]:
      for NS_EXP in [0.3, 0.8]:
       for NEGATIVE in [5]:
        start_t = time.time()
        n2v = graph2vec.Node2Vec(
            walklen=WALKLEN,
            epochs=EPOCH,
            return_weight=RETURN_WEIGHT,
            neighbor_weight=NEIGHBOR_WEIGHT,
            w2vparams={'window': WINDOW,
                       'size': EMBED_SIZE, 
                       'negative': NEGATIVE, 
                       'iter': 5,
                       'ns_exponent': NS_EXP,
                       'batch_words': 128}
        )
        n2v.fit(G, verbose=False)

        # Make embedding result matrix
        nodes = list(G.nodes)
        embeds = np.empty((len(nodes), EMBED_SIZE))
        for i in nodes:
            embeds[i] = n2v.predict(i)

        # Density based methods don't seem to perform well
        # Since we know ahead of time the nb of clusters
        # Agglomerative is best performing and has stable results
        agglo = cluster.AgglomerativeClustering(
            n_clusters=n_clusters, 
            affinity='cosine', # euclidean with Ward is slightly worse
            linkage='average'
        ).fit(embeds).labels_

        x = evalClusteringOnLabels(agglo, labels)
        print("-------------------\n\n")

        res = res.append({
            'method': 'Node2Vec', 
            'params':{
                'walklen': WALKLEN,
                'epochs': EPOCH,
                'return_weight': RETURN_WEIGHT,
                'neighbor_weight': NEIGHBOR_WEIGHT,
                'window': WINDOW,
                'size': EMBED_SIZE, 
                'negative': NEGATIVE, 
                'iter': EPOCH,
                'ns_exponent': NS_EXP,
                'batch_words': 128,
            }, 
            'traintime': time.time() - start_t, 
            'MI':x[0], 
            'RAND':x[1], 
            'F-M':x[2]}, 
            ignore_index=True)

C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   -0.00
adj. RAND score: -0.00
F-M score:       0.20
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.17
adj. RAND score: 0.04
F-M score:       0.21
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


...

C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.63
adj. RAND score: 0.51
F-M score:       0.54
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


adj. MI score:   0.63
adj. RAND score: 0.51
F-M score:       0.54
-------------------


adj. MI score:   0.65
adj. RAND score: 0.54
F-M score:       0.56
-------------------




C:\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


In [8]:
res['nodes'] = len(G.nodes)
res['edges'] = len(G.edges)
res['clusters'] = n_clusters

res.to_csv(f'RESULTS_{int(time.time())}.csv')